# 形成label数据

In [69]:
import os
import pandas as pd
import shutil
from glob import glob

root = r'D:/20240420-LiXiaoFeng/data'
label_data = []
img_dir = os.path.join(root, 'images')
os.makedirs(img_dir, exist_ok=True)
msk_dir = os.path.join(root, 'masks')
os.makedirs(msk_dir, exist_ok=True)

for subset in ['train', 'test']:
    for label in os.listdir(os.path.join(root, subset)):
        if '-' in label:
            lv = 0
        else:
            lv = 1
        for sample in os.listdir(os.path.join(root, subset, label)):
#             print(sample)
            if os.path.isdir(os.path.join(root, subset, label, sample)):
                label_data.append([sample.replace(',', ''), lv, subset])
#                 mp, ip = list(sorted(glob(os.path.join(root, subset, label, sample, '*.nii.gz')), key=lambda x: os.path.getsize(x)))
#                 shutil.copy(os.path.join(root, subset, label, sample, mp), os.path.join(msk_dir, f"{sample}.nii.gz"))
#                 shutil.copy(os.path.join(root, subset, label, sample, ip), os.path.join(img_dir, f"{sample}.nii.gz"))
label_data = pd.DataFrame(label_data, columns=['ID', 'label', 'group']).drop_duplicates('ID')
label_data['ID'] = label_data['ID'].map(lambda x: f"{x}.nii.png")
label_data = pd.merge(label_data, pd.DataFrame(os.listdir(r'D:/20240420-LiXiaoFeng/vnet_crop_full/'), columns=['ID']), on='ID', how='inner')
label_data['ID'] = label_data['ID'].map(lambda x: x.replace('.png', '.gz'))
label_data.to_csv(os.path.join(root, 'label.csv'), index=False)
label_data

,ID,label,group
0,CAI_CONG_ZHI_P00581652.nii.gz,1,train
1,CAI_XUE_YING_P00359999.nii.gz,1,train
2,CAO_BAO_HUA_P00190203.nii.gz,1,train
3,CAO_MIN_P00549794.nii.gz,1,train
4,CHEN_LAN_YING_P00551393.nii.gz,1,train
...,...,...,...
284,ZHAO_YUN_P00310065.nii.gz,0,test
285,ZHAO_ZHANG_ZHI_P00206369.nii.gz,0,test
286,ZHENG_XI_DE_P00266862.nii.gz,0,test
287,ZHOU_RONG_P00306340.nii.gz,0,test


In [70]:
label_data['group'].value_counts()

train    209
test      80
Name: group, dtype: int64

In [71]:
import pandas as pd
import os
import numpy as np
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd      

train_data = label_data[label_data['group'] == 'train']
test_data = label_data[label_data['group'] != 'train']
ds = split_dataset4sol(train_data, y_data=train_data['label'], n_trails=5, save_dir='.', 
                       random_state=0, cv=False, test_size=0.3)
for idx, (train, val) in enumerate(ds):
    val['group'] = 'val'
    pd.concat([train, val, test_data], axis=0).to_csv(os.path.join('split_info', f'label-RND-{idx}.csv'), index=False)
    train['ID'] = train['ID'].map(lambda x: x.replace('.gz', '.png'))
    train[['ID', 'label']].to_csv(os.path.join('split_info', f'train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    valtest = pd.concat([val, test_data], axis=0)
    valtest['ID'] = valtest['ID'].map(lambda x: x.replace('.gz', '.png'))
    valtest[['ID', 'label']].to_csv(os.path.join('split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')
    print(f"随机划分：{idx}，训练集：{train.shape[0]}, 测试集集：{valtest.shape[0]}")

随机划分：0，训练集：146, 测试集集：143
随机划分：1，训练集：146, 测试集集：143
随机划分：2，训练集：146, 测试集集：143
随机划分：3，训练集：146, 测试集集：143
随机划分：4，训练集：146, 测试集集：143


In [74]:
from onekey_algo.custom.utils import print_join_info
label_data = pd.read_csv('split_info/label-RND-1.csv')
label_data['ID'] = label_data['ID'].map(lambda x: x.replace('.gz', '.png'))
label_data = pd.merge(label_data, pd.DataFrame(os.listdir(r'D:/20240420-LiXiaoFeng/vnet_crop_full/'), columns=['ID']), on='ID', how='inner')
print_join_info(label_data, pd.DataFrame(os.listdir(r'D:/20240420-LiXiaoFeng/vnet_crop_full/'), columns=['ID']))
label_data.to_csv('data/group.csv', index=False)
label_data
# 

[2024-04-22 18:09:09 - __init__.py:  55]	INFO	ID特征完全匹配！


,ID,label,group
0,DONG HUIP00343158.nii.png,0,train
1,YAO YU MING_P00096109.nii.png,0,train
2,DENG YUE YINGP00332485.nii.png,0,train
3,JIANG XIU JUAN_P00104388.nii.png,0,train
4,DAI_XIAO_HUI_P00307303.nii.png,0,train
...,...,...,...
284,ZHAO_YUN_P00310065.nii.png,0,test
285,ZHAO_ZHANG_ZHI_P00206369.nii.png,0,test
286,ZHENG_XI_DE_P00266862.nii.png,0,test
287,ZHOU_RONG_P00306340.nii.png,0,test


In [75]:
label_data['group'].value_counts()

train    146
test      80
val       63
Name: group, dtype: int64

In [76]:
label_data[label_data['group'] == 'train'][['ID', 'label']].to_csv('split_info/train.txt', sep='\t', index=False, header=False)
label_data[label_data['group'] != 'train'][['ID', 'label']].to_csv('split_info/test.txt', sep='\t', index=False, header=False)

In [26]:
import shutil

for r, ds, fs in os.walk(r'D:/20240420-LiXiaoFeng/'):
    for f in fs:
        shutil.move(os.path.join(r, f), os.path.join(r, f.replace(',', '')))